In [2]:
import arcpy
from arcpy import env
from arcpy.da import *
from arcpy.sa import *
import os
import math

In [3]:
dem_path = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb\DEM"
save_path = r"D:\ArcGISProjects\workspace\duyun\featuressdata_dy.gdb"

In [4]:
env.workspace = save_path

In [5]:
raster_list = arcpy.ListRasters()
print(raster_list)

['BIO', 'PRE', 'SRA', 'TMP', 'VAP', 'WIN', 'DEM', 'NDVI', 'TDQS', 'LIGHT', 'LON', 'LAT', 'SLOPE', 'ASP', 'CUR', 'TWI3', 'TWI5', 'TPI3', 'TPI5']


In [16]:
# 基于DEM数据生成新数据的函数

# 取消并行处理
# 取消并行处理
def disable_parallel_processing(func):
    def wrapper(*args, **kwargs):
        with arcpy.EnvManager(parallelProcessingFactor="0"):
            return func(*args, **kwargs)
    return wrapper

# 计算坡度
@disable_parallel_processing
def get_slope(dem_path):
    with arcpy.EnvManager(parallelProcessingFactor="0"):
        slope_raster = arcpy.sa.Slope(dem_path, "DEGREE", 1, "PLANAR", "METER")
        return slope_raster

# 计算坡向
@disable_parallel_processing
def get_asp(dem_path):
    with arcpy.EnvManager(parallelProcessingFactor="0"):
        aspect_raster = arcpy.sa.Aspect(
            dem_path, "PLANAR", "METER", "GEODESIC_AZIMUTHS"
        )
        return aspect_raster
# 计算曲率
@disable_parallel_processing
def get_cur(dem_path):
    with arcpy.EnvManager(parallelProcessingFactor="0"):
        curvature_raster = arcpy.sa.Curvature(dem_path, 1, None, None)
        return curvature_raster
# 坡度(弧度)
@disable_parallel_processing
def get_rad_slope(dem_raser):
    """得到为弧度的坡度"""
    slope_raster = Slope(dem_raser,"DEGREE",1,"PLANAR","METER")
    result_raster = Con(slope_raster==0,0.001, Tan(slope_raster *math.pi/180 ))
    return result_raster
# 填洼
@disable_parallel_processing
def fill_dem(dem_raster):
    """参数默认为最佳"""
    fill_raster = Fill(dem_raster)
    return fill_raster

# 流向
@disable_parallel_processing
def flowdir_dem(dem_raster,dir_index):
    """"D8算法,dir_index{1:正常流动,0:强制外向流动}"""
    if dir_index:
        flowdir_raster = FlowDirection(dem_raster,"NORMAL",None,"D8")
        return flowdir_raster
    else:
        flowdir_raster = FlowDirection(dem_raster,"FORCE", None, "D8")
        return flowdir_raster
    
# 流量(+1)
@disable_parallel_processing
def acc_dem(dem_raster,cell_size):
    """输入流向类型D8"""
    acc_raster = FlowAccumulation(dem_raster,None,"FLOAT","D8")
    result_raster = (acc_raster+1)*cell_size
    return result_raster

# 单位面积汇流量

# 计算TPI
@disable_parallel_processing
def calc_tpi(dem_raster,focous_size):
    """dem_raster:DEM数据,focous_size:窗口大小"""
    # 焦点统计
    focous_raster = FocalStatistics(dem_raster,NbrRectangle(focous_size,focous_size,"CELL"),"MEAN","DATA")
    # 结果输出
    result_raster = dem_raster-focous_raster
    return result_raster


# 计算SCA（单位面积的汇流量）
def calc_sca(flow_acc,flow_dir,cell_size):
    cell_area = cell_size*cell_size
    result_raster = Con(flow_acc==0,cell_area,flow_acc)*cell_area/Con((flow_dir==1)|(flow_dir==8)|(flow_dir==32)|(flow_dir==64),cell_size,cell_area)
    return result_raster

# 计算TWI
@disable_parallel_processing
def calc_twi(dem_raster,cell_size):
    """dem_raster:DEM数据"""
    # 得到正切坡度
    asp_raster = get_rad_slope(dem_raster)
    print("asp done!")
    # 流向
    fd_raster = flowdir_dem(dem_raster,1)
    print("fd done!")
    # sca流量
    ac_raster = acc_dem(fd_raster,cell_size)
    print("acc done!")
    # 计算TWI  
    result_raster = Ln(ac_raster/asp_raster)
    return result_raster


# 焦点统计计算
@disable_parallel_processing
def calc_foc(dem_raster, size):
    nbrhood = NbrRectangle(size, size, "CELL")
    out_raster = FocalStatistics(dem_raster, nbrhood, "MEAN", "DATA")
    return out_raster


# 相对位置计算
@disable_parallel_processing
def calc_tpi(dem_raster, size):
    foc_raster = calc_foc(dem_raster, size)
    # tpi_raster = (dem_raster - foc_raster)
    tpi_raster = (foc_raster / dem_raster) * 100
    return tpi_raster

In [12]:
# 得到坡度
get_slope(dem_path).save("SLOPE")

In [13]:
# 得到坡向
get_asp(dem_path).save("ASP")

In [14]:
# 得到曲率
get_cur(dem_path).save("CUR")

In [15]:
# 得到地形湿度指数（TWI）
calc_twi(dem_path,3).save("TWI3")
calc_twi(dem_path,5).save("TWI5")

asp done!
fd done!
acc done!
asp done!
fd done!
acc done!


In [17]:
# 得到相对位置
calc_tpi(dem_path,3).save("TPI3")
calc_tpi(dem_path,5).save("TPI5")
